# Learning to Rank Demo | Activate 2020

## Overview

In this notebook, we will train a Learning to Rank model from user click data using ml4ir.

#### Key Takeaways
- How to install and get started with ml4ir as a script and a library
- Defining a ranking pipeline from scratch
- Transfer learning for ml4ir models
- Serving models trained on ml4ir

#### Learning to Rank
The goal of Learning to Rank(LTR) is to come up with a ranking function to generate an optimal ordering of a list of documents. In this notebook, we will learn a simple pointwise ranking function using a listwise loss which will predict the ranking scores for all records of a given query. These scores can then be used at inference to determine the optimal ordering.

## Contents

1. [Install ml4ir](#Install-ml4ir)
1. [Look at the Data](#Look-at-the-Data)
1. [Define the FeatureConfig](#Define-the-FeatureConfig)
1. [Define the ModelConfig](#Define-the-ModelConfig)
1. [Using ml4ir as a script](#Using-ml4ir-as-a-script)
1. [Using ml4ir as a library](#Using-ml4ir-as-a-library)
    1. [Setup](#Setup)
    1. [Load the FeatureConfig](#Load-the-FeatureConfig)
    1. [Create a RelevanceDataset](#Create-a-RelevanceDataset)
    1. [Define an InteractionModel](#Define-an-InteractionModel)
    1. [Define losses, metrics and optimizer](#Define-losses,-metrics-and-optimizer)
    1. [Define the scoring function, or the Scorer](#Define-a-scoring-function,-or-the-Scorer)
    1. [Combine it all to create a RankingModel](#Combine-it-all-to-create-a-RankingModel)
    1. [Train and Evaluate your RankingModel](#Train-and-Evaluate-your-RankingModel)
    1. [Save the trained RankingModel](#Save-the-trained-RankingModel)
1. [Let's try some Transfer Learning](#Let's-try-some-Transfer-Learning)
    1. [What does ml4ir save?](#What-does-ml4ir-save?)
    1. [Using pre-trained character embeddings](#Using-pre-trained-character-embeddings)
1. [Model Serving](#Model-Serving)
    1. [JVM Serving Logic](#JVM-Serving-Logic)
    1. [Serving your Ranking Model](#Serving-your-Ranking-Model)

## Install ml4ir

In [1]:
!pip install ml4ir

Looking in indexes: https://pypi.python.org/simple


You should consider upgrading via the '/Users/ashish.srinivasa/search_relevance/ml4ir/python/env/.ranking_venv3/bin/python3 -m pip install --upgrade pip' command.


## Look at the data

In [2]:
import pandas as pd

df_train = pd.read_csv("../ml4ir/applications/ranking/tests/data/csv/train/file_0.csv")
df_train.head(7)

,query_id,rank,query_text,domain_name,text_match_score,page_views_score,quality_score,name_match,domain_id,clicked
0,query_0,1,UQHA3QP4ZVO,domain_0,1.101297,0.002044,0.00000,0,0,1
1,query_0,2,UQHA3QP4ZVO,domain_0,0.380570,0.004078,0.30103,0,0,0
2,query_1,1,8M3NWYX4E6I,domain_1,1.024334,0.008686,0.30103,1,1,1
3,query_1,2,8M3NWYX4E6I,domain_1,0.821515,0.200264,0.00000,1,1,0
4,query_1,3,8M3NWYX4E6I,domain_1,0.821323,0.200264,0.00000,0,1,0
5,query_1,4,8M3NWYX4E6I,domain_1,0.821515,0.200264,0.00000,0,1,0
6,query_1,5,8M3NWYX4E6I,domain_1,0.821323,0.200264,0.00000,1,1,0


## Define the FeatureConfig

**YAML File** -> configs/activate_2020/feature_config.yaml



| Feature          | Type    | TFRecord Type | Usage                                    |
| ---------------- | -------- | ------------- | ---------------------------------------- |
| query_text       | Text     | Context       | Character Embeddings -> biLSTM Encoding  |
| domain_name      | Text     | Context       | VocabLookup -> Categorical Embedding     |
| text_match_score | Numeric  | Sequence      | float                                    |
| page_views_score | Numeric  | Sequence      | float                                    |
| quality_score    | Numeric  | Sequence      | float                                    |

## Define the ModelConfig

In [3]:
print(open("configs/activate_2020/model_config.yaml").read())

architecture_key: dnn
layers:
  - type: dense
    name: first_dense
    units: 256
    activation: relu
  - type: dropout
    name: first_dropout
    rate: 0.3
  - type: dense
    name: second_dense
    units: 64
    activation: relu
  - type: dense
    name: final_dense
    units: 1
    activation: null



## Using ml4ir as a script

In [4]:
!python ../ml4ir/applications/ranking/pipeline.py \
--data_format csv \
--data_dir ../ml4ir/applications/ranking/tests/data/csv \
--feature_config configs/activate_2020/feature_config.yaml \
--model_config configs/activate_2020/model_config.yaml \
--execution_mode train_inference_evaluate \
--loss_key rank_one_listnet \
--num_epochs 3 \
--models_dir ../models/activate_2020 \
--logs_dir ../logs/activate_2020 \
--run_id activate_demo

INFO: 2020-09-11 04:16:30.487 
Logging initialized. Saving logs to : ../logs/activate_2020/activate_demo
INFO: 2020-09-11 04:16:30.487 
Run ID: activate_demo
DEBUG: 2020-09-11 04:16:30.487 
CLI args: 
{
    "data_dir": "../ml4ir/applications/ranking/tests/data/csv",
    "data_format": "csv",
    "tfrecord_type": "sequence_example",
    "feature_config": "configs/activate_2020/feature_config.yaml",
    "model_file": "",
    "model_config": "configs/activate_2020/model_config.yaml",
    "optimizer_key": "adam",
    "loss_key": "rank_one_listnet",
    "metrics_keys": "['MRR', 'ACR']",
    "monitor_metric": "new_MRR",
    "monitor_mode": "max",
    "num_epochs": 3,
    "batch_size": 128,
    "learning_rate": 0.01,
    "learning_rate_decay": 0.9,
    "learning_rate_decay_steps": 1000,
    "compute_intermediate_stats": true,
    "execution_mode": "train_inference_evaluate",
    "random_state": 123,
    "run_id": "activate_demo",
    "run_group": "general",
    "run_notes": "",
    "models_di

INFO: 2020-09-11 04:16:30.549 
Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train/file_0.tfrecord
INFO: 2020-09-11 04:16:31.631 
1 files found under ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train
INFO: 2020-09-11 04:16:34.374 
Created TFRecordDataset from SequenceExample protobufs from 1 files : ['../ml4ir/applications/ranking/tests/data/csv/tfr
INFO: 2020-09-11 04:16:34.374 
1 files found under ../ml4ir/applications/ranking/tests/data/csv/validation
INFO: 2020-09-11 04:16:34.375 
Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/validation/file_0.csv, ..
INFO: 2020-09-11 04:16:34.375 
Loading dataframe from path : ../ml4ir/applications/ranking/tests/data/csv/validation/file_0.csv
INFO: 2020-09-11 04:16:34.387 
Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/validation/file_0.tfrecord
INFO: 2020-09-11 04:16:35.697 
1 files found under ../ml4ir/applications/ranking/

INFO: 2020-09-11 04:16:42.666 
Training Model
INFO: 2020-09-11 04:16:42.670 
Starting Epoch : 1
INFO: 2020-09-11 04:16:42.670 
{}
Epoch 1/3
INFO: 2020-09-11 04:16:49.597 
[epoch: 1 | batch: 0] {'batch': 0, 'size': 128, 'loss': 2.0728323, 'old_MRR': 0.8084635, 'new_MRR': 0.56783265, 'old_ACR': 1.5859375, 'new_ACR': 2.4765625}
     11/Unknown - 9s 814ms/step - loss: 1.8710 - old_MRR: 0.7875 - new_MRR: 0.6783 - old_ACR: 1.6491 - new_ACR: 2.0291INFO: 2020-09-11 04:16:51.901 
Evaluating Model
INFO: 2020-09-11 04:16:55.088 
Completed evaluating model
INFO: 2020-09-11 04:16:55.088 
None

Epoch 00001: val_new_MRR improved from -inf to 0.70944, saving model to ../models/activate_2020/activate_demo/checkpoint.tf
INFO: 2020-09-11 04:17:27.015 
End of Epoch 1
INFO: 2020-09-11 04:17:27.015 
{'loss': 1.8709832213141702, 'old_MRR': 0.7874729, 'new_MRR': 0.6783062, 'old_ACR': 1.6491477, 'new_ACR': 2.0291193, 'val_loss': 1.8305743824351917, 'val_old_MRR': 0.7827933, 'val_new_MRR': 0.709439, 'val_old_AC

## Using ml4ir as a library

<img src="images/model_framework.png" alt="ml4ir Architecture" style="width: 500px;" align="center"/>

### Setup

In [5]:
MODEL_CONFIG_PATH = "configs/activate_2020/model_config.yaml"
FEATURE_CONFIG_PATH = "configs/activate_2020/feature_config.yaml"

DATA_DIR = "../ml4ir/applications/ranking/tests/data/csv"
MODELS_DIR = '../models/activate_2020'
LOGS_DIR = '../logs/activate_2020'

MAX_SEQUENCE_SIZE = 25

In [6]:
import logging
import tensorflow as tf
import os

from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import Optimizer
from typing import List, Union, Type

from ml4ir.base.io.local_io import LocalIO
from ml4ir.base.io.file_io import FileIO
from ml4ir.base.config.keys import *
from ml4ir.base.data.relevance_dataset import RelevanceDataset
from ml4ir.base.features.feature_config import FeatureConfig, SequenceExampleFeatureConfig
from ml4ir.base.model.scoring.scoring_model import RelevanceScorer
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.model.losses.loss_base import RelevanceLossBase
from ml4ir.base.model.optimizer import get_optimizer
from ml4ir.applications.ranking.model.ranking_model import RankingModel
from ml4ir.applications.ranking.config.keys import LossKey, MetricKey, ScoringTypeKey
from ml4ir.applications.ranking.model.losses import loss_factory
from ml4ir.applications.ranking.model.metrics import metric_factory

In [7]:
# Set up file I/O handler
file_io : FileIO = LocalIO()
    
# Create directories for models and logs
file_io.make_directory(LOGS_DIR, clear_dir=True)
file_io.make_directory(MODELS_DIR, clear_dir=True)

# Set up logger
logger = logging.getLogger()

tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### Load the FeatureConfig

In [8]:
feature_config: SequenceExampleFeatureConfig = FeatureConfig.get_instance(
    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
    feature_config_dict=file_io.read_yaml(FEATURE_CONFIG_PATH),
    logger=logger)
print("Training features\n-----------------")
print("\n".join(feature_config.get_train_features(key="name")))

Training features
-----------------
text_match_score
page_views_score
quality_score
query_text
domain_name


### Create a RelevanceDataset

In [9]:
ranking_dataset = RelevanceDataset(data_dir=DATA_DIR,
                            data_format=DataFormatKey.CSV,
                            feature_config=feature_config,
                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                            max_sequence_size=MAX_SEQUENCE_SIZE,
                            batch_size=128,
                            preprocessing_keys_to_fns={},
                            file_io=file_io,
                            logger=logger)

### Define an InteractionModel

In [10]:
interaction_model: InteractionModel = UnivariateInteractionModel(
                                            feature_config=feature_config,
                                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                            max_sequence_size=MAX_SEQUENCE_SIZE,
                                            feature_layer_keys_to_fns={},
                                            file_io=file_io,
                                        )

### Define losses, metrics and optimizer
##### Use predefined losses, metrics and optimizers or create your own!

In [11]:
# Define loss object from loss key
loss: RelevanceLossBase = loss_factory.get_loss(
                                loss_key=LossKey.RANK_ONE_LISTNET,
                                scoring_type=ScoringTypeKey.POINTWISE)
    
# Define metrics objects from metrics keys
metric_keys = [MetricKey.MRR, MetricKey.ACR]
metrics: List[Union[Type[Metric], str]] = [metric_factory.get_metric(metric_key=m) for m in metric_keys]
    
# Define optimizer
optimizer: Optimizer = get_optimizer(
                            optimizer_key=OptimizerKey.ADAM,
                            learning_rate=0.001
                        )

### Define a scoring function, or the Scorer

In [12]:
scorer: RelevanceScorer = RelevanceScorer.from_model_config_file(
    model_config_file=MODEL_CONFIG_PATH,
    interaction_model=interaction_model,
    loss=loss,
    logger=logger,
    file_io=file_io,
)

### Combine it all to create a RankingModel

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Logger is initialized...")

ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    file_io=file_io,
                                    logger=logger,
                                )

DEBUG:root:Logger is initialized...
INFO:root:Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query_text (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_DecodePaddedRaw (Te [(None, 1, 20)]      0           query_text[0][0]                 
__________________________________________________________________________________________________
domain_name (InputLayer)        [(None, 1)]          0                                            
________________________________________________

### Train and Evaluate your RankingModel

In [14]:
ranking_model.fit(dataset=ranking_dataset,
                  num_epochs=3, 
                  models_dir=MODELS_DIR,
                  logs_dir=LOGS_DIR,
                  monitor_metric="new_MRR",
                  monitor_mode="max")

INFO:root:Training Model
INFO:root:Starting Epoch : 1
INFO:root:{}


Epoch 1/3


INFO:root:[epoch: 1 | batch: 0] {'batch': 0, 'size': 128, 'loss': 2.0692837, 'old_MRR': 0.8084635, 'new_MRR': 0.5622395, 'old_ACR': 1.5859375, 'new_ACR': 2.625}


     11/Unknown - 8s 767ms/step - loss: 1.9714 - old_MRR: 0.7875 - new_MRR: 0.6697 - old_ACR: 1.6491 - new_ACR: 2.0866

INFO:root:Evaluating Model
INFO:root:Completed evaluating model
INFO:root:None


...

INFO:root:Evaluating Model
INFO:root:Completed evaluating model
INFO:root:None



Epoch 00003: val_new_MRR improved from 0.70686 to 0.70921, saving model to ../models/activate_2020/checkpoint.tf


INFO:root:End of Epoch 3
INFO:root:{'loss': 1.8562179045243696, 'old_MRR': 0.7874729, 'new_MRR': 0.69355386, 'old_ACR': 1.6491477, 'new_ACR': 1.9879261, 'val_loss': 1.8498522368344394, 'val_old_MRR': 0.7827933, 'val_new_MRR': 0.709209, 'val_old_ACR': 1.6491477, 'val_new_ACR': 1.9602273}


11/11 [==============================] - 35s 3s/step - loss: 1.8562 - old_MRR: 0.7875 - new_MRR: 0.6936 - old_ACR: 1.6491 - new_ACR: 1.9879 - val_loss: 1.8499 - val_old_MRR: 0.7828 - val_new_MRR: 0.7092 - val_old_ACR: 1.6491 - val_new_ACR: 1.9602


INFO:root:Completed training model
INFO:root:None


In [15]:
ranking_model.predict(test_dataset=ranking_dataset.test).sample(10)

,query_id,clicked,name_match,query_text,domain_name,rank,score,new_rank
364,b'query_1080',0,0.0,b'171fs8se',b'domain_0',2,0.219567,2
63,b'query_1010',0,0.0,b'9u4nel0m',b'domain_0',2,0.471660,2
4,b'query_224',0,0.0,b'u7bxar',b'domain_4',2,0.122372,3
301,b'query_530',0,0.0,b'4l7e19mc',b'domain_0',1,0.526122,1
331,b'query_189',0,1.0,b'i1mpco01ec1prsy',b'domain_4',5,0.170475,3
57,b'query_1008',0,1.0,b'97vajsx',b'domain_3',2,0.194165,5
350,b'query_543',0,1.0,b'loqoca9a9vmib',b'domain_3',3,0.437938,1
323,b'query_1308',0,0.0,b'clhghwwsz5lmxf',b'domain_3',6,0.154077,5
196,b'query_153',1,0.0,b'vkv5ochbzn4l',b'domain_3',1,0.514826,1
409,b'query_56',0,1.0,b'014vesb',b'domain_1',1,0.285812,1


In [16]:
import pandas as pd

pd.DataFrame(ranking_model.evaluate(test_dataset=ranking_dataset.test)[0], columns=["value"])

INFO:root:Overall Metrics: 
query_count        1408.000000
old_ACR               1.650568
new_ACR               1.965909
old_MRR               0.785881
new_MRR               0.698818
perc_improv_ACR     -19.104991
perc_improv_MRR     -11.078333
dtype: float64


,value
query_count,1408.000000
old_ACR,1.650568
new_ACR,1.965909
old_MRR,0.785881
new_MRR,0.698818
perc_improv_ACR,-19.104991
perc_improv_MRR,-11.078333


### Save the trained RankingModel

In [17]:
ranking_model.save(models_dir=MODELS_DIR,
                   preprocessing_keys_to_fns={},
                   required_fields_only=True)

INFO:root:Final model saved to : ../models/activate_2020/final


--------

## Let's try some Transfer Learning

### What does ml4ir save?

<img src="images/ml4ir_savedmodel.png" alt="ml4ir Architecture" style="width: 350px;" align="left"/>

### Using pre-trained character embeddings

In [18]:
initialize_layers_dict = {
    "query_text_bytes_embedding" : "models/activate_2020/bytes_embedding.npy"
}
freeze_layers_list = ["query_text_bytes_embedding"]
ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    initialize_layers_dict=initialize_layers_dict,
                                    freeze_layers_list=freeze_layers_list,
                                    file_io=file_io,
                                    logger=logger,
                                )

INFO:root:Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query_text (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_DecodePaddedRaw_1 ( [(None, 1, 20)]      0           query_text[0][0]                 
__________________________________________________________________________________________________
domain_name (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________

INFO:root:Setting query_text_bytes_embedding weights from models/activate_2020/bytes_embedding.npy
INFO:root:Freezing query_text_bytes_embedding layer


In [19]:
import numpy as np

ranking_model_embeddings = ranking_model.model.get_layer("query_text_bytes_embedding").get_weights()
query_classification_embeddings = np.load("models/activate_2020/bytes_embedding.npy")

assert tf.reduce_any(tf.equal(ranking_model_embeddings, query_classification_embeddings))

print(ranking_model_embeddings)

[array([[-0.03738469,  0.00727513, -0.02006867, ...,  0.01078511,
        -0.028496  , -0.04102874],
       [ 0.00512887,  0.0062821 , -0.0010671 , ...,  0.04945388,
        -0.0132054 , -0.01177131],
       [-0.00937623, -0.03438247,  0.00176773, ...,  0.046078  ,
        -0.0310035 , -0.04288797],
       ...,
       [-0.04410168,  0.0383402 ,  0.03348425, ...,  0.02123589,
         0.02240864, -0.04049417],
       [ 0.03601265,  0.04585798,  0.00272902, ..., -0.00353998,
        -0.04783431,  0.02852656],
       [ 0.04903785, -0.03518286,  0.00195389, ...,  0.03783921,
        -0.01398294,  0.0107099 ]], dtype=float32)]


---------

## Model Serving

### JVM Serving Logic


```
def runQueriesAgainstDocs(
        csvDataPath: String,
        modelPath: String,
        featureConfigPath: String,
        inputTFNode: String,
        scoresTFNode: String): Iterable[(StringMapQueryContextAndDocs, SequenceExample, Array[Float])] = {
  
  val featureConfig = ModelFeaturesConfig.load(featureConfigPath)
  val sequenceExampleBuilder = StringMapSequenceExampleBuilder.withFeatureProcessors(featureConfig)
  val rankingModelConfig = ModelExecutorConfig(inputTFNode, scoresTFNode)
  val rankingModel = new SavedModelBundleExecutor(modelPath, rankingModelConfig)

  val queryContextsAndDocs = StringMapCSVLoader.loadDataFromCSV(csvDataPath, featureConfig)

  queryContextsAndDocs.map {
    case q @ StringMapQueryContextAndDocs(queryContext, docs) =>
      val sequenceExample = sequenceExampleBuilder.build(queryContext, docs)
      (q, sequenceExample, rankingModel(sequenceExample))
  }
}

val allScores: Iterable[
  (StringMapQueryContextAndDocs, SequenceExample, Array[Float])] = runQueriesAgainstDocs(
    pathFor("test_data.csv"),
    pathFor("ranking_model_bundle"),
    pathFor("feature_config.yaml"),
    "serving_tfrecord_protos",
    "ranking_score"
  )
```

### Serving your RankingModel

In [20]:
!cd ../../jvm; mvn test -Dtest=ml4ir.inference.tensorflow.TensorFlowInferenceTest

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for ml4ir:ml4ir-inference:jar:0.0.2-SNAPSHOT
[WARNING] 'build.plugins.plugin.version' for org.apache.maven.plugins:maven-compiler-plugin is missing. @ line 87, column 15
[WARNING] 'build.plugins.plugin.version' for org.apache.maven.plugins:maven-jar-plugin is missing. @ line 94, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] ml4ir-parent                                                       [pom]
[INFO] ml4ir-inference                                                    [jar]
[INFO] 
[INFO] -------------------------< ml4ir:ml4ir-parent >-----


Results :

Tests run: 1, Failures: 0, Errors: 0, Skipped: 0

[INFO] ------------------------------------------------------------------------
[INFO] Reactor Summary for ml4ir-parent 0.0.2-SNAPSHOT:
[INFO] 
[INFO] ml4ir-parent ....................................... SUCCESS [  0.004 s]
[INFO] ml4ir-inference .................................... SUCCESS [ 37.924 s]
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  38.264 s
[INFO] Finished at: 2020-09-11T04:23:57-07:00
[INFO] ------------------------------------------------------------------------
